In [24]:
from collections import Counter, defaultdict
from itertools import combinations
import pandas as pd
import numpy as np
import operator
import math
import itertools
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

In [25]:
df = pd.read_csv('default of credit card clients.csv', header=1)
df = df.fillna(df.median())
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [26]:
ohe_df = pd.get_dummies(df, columns=['SEX', 'EDUCATION', 'MARRIAGE'], dtype=int)
ohe_df.head()


,ID,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3
0,1,20000,24,2,2,-1,-1,-2,-2,3913,...,0,1,0,0,0,0,0,1,0,0
1,2,120000,26,-1,2,0,0,0,2,2682,...,0,1,0,0,0,0,0,0,1,0
2,3,90000,34,0,0,0,0,0,0,29239,...,0,1,0,0,0,0,0,0,1,0
3,4,50000,37,0,0,0,0,0,0,46990,...,0,1,0,0,0,0,0,1,0,0
4,5,50000,57,-1,0,-1,0,0,0,8617,...,0,1,0,0,0,0,0,1,0,0


In [27]:
def normalize(x):
    return (x - np.mean(x)) / np.std(x)

ohe_df.loc[:,'BILL_AMT1':'PAY_AMT6'] = ohe_df.loc[:,'BILL_AMT1':'PAY_AMT6'].apply(normalize, axis=1)
ohe_df[['LIMIT_BAL', 'AGE']] = ohe_df[[
    'LIMIT_BAL', 'AGE']].apply(normalize)
ohe_df

,ID,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3
0,1,-1.136720,-1.246020,2,2,-1,-1,-2,-2,2.488409,...,0,1,0,0,0,0,0,1,0,0
1,2,-0.365981,-1.029047,-1,2,0,0,0,2,0.707157,...,0,1,0,0,0,0,0,0,1,0
2,3,-0.597202,-0.161156,0,0,0,0,0,0,2.316935,...,0,1,0,0,0,0,0,0,1,0
3,4,-0.905498,0.164303,0,0,0,0,0,0,1.364748,...,0,1,0,0,0,0,0,1,0,0
4,5,-0.905498,2.334029,-1,0,-1,0,0,0,-0.450997,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,0.404759,0.381275,0,0,0,0,0,0,1.563337,...,0,0,1,0,0,0,0,1,0,0
29996,29997,-0.134759,0.815221,-1,-1,-1,-1,0,0,-0.413477,...,0,0,1,0,0,0,0,0,1,0
29997,29998,-1.059646,0.164303,4,3,2,-1,0,0,-0.562441,...,0,1,0,0,0,0,0,0,1,0
29998,29999,-0.674276,0.598248,1,-1,0,0,0,-1,-1.088271,...,0,0,1,0,0,0,0,1,0,0


In [28]:
X = ohe_df.drop(columns=['default payment next month', 'ID']).dropna()
Y = ohe_df.dropna()['default payment next month']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

In [29]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(100),
                      activation='logistic',
                      max_iter=200,
                      solver='lbfgs',
                      verbose=True,
                      random_state=42)
mlp_clf.fit(Xtrain, Ytrain)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         3501     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.94859D-01    |proj g|=  4.22349D-01
 This problem is unconstrained.

At iterate    1    f=  5.23119D-01    |proj g|=  2.20378D-02

At iterate    2    f=  5.21011D-01    |proj g|=  1.54833D-02

At iterate    3    f=  5.17478D-01    |proj g|=  1.99436D-02

At iterate    4    f=  5.12050D-01    |proj g|=  3.20470D-02

At iterate    5    f=  4.98852D-01    |proj g|=  5.32001D-02

At iterate    6    f=  4.83584D-01    |proj g|=  4.86146D-02

At iterate    7    f=  4.65736D-01    |proj g|=  5.10391D-03

At iterate    8    f=  4.62603D-01    |proj g|=  2.11012D-02

At iterate    9    f=  4.57305D-01    |proj g|=  2.01771D-02

At iterate   10    f=  4.49183D-01    |proj g|=  2.59244D-02

At iterate   11    f=  4.46320D-01    |proj g|=  1.75615D-02

At iterate   12    f=  4.44867D-01    |proj g|=  4.26015

MLPClassifier(activation='logistic', hidden_layer_sizes=100, random_state=42,
              solver='lbfgs', verbose=True)

In [30]:
Ypred = mlp_clf.predict(Xtest)
Ypred_prob = mlp_clf.predict_proba(Xtest)[:, 1]

# Print Results
print("Classification Report (MLP Classifier):")
print(classification_report(Ytest, Ypred))

print("\nConfusion Matrix (MLP Classifier):")
print(confusion_matrix(Ytest, Ypred))

print("\nROC-AUC Score (MLP Classifier):")
print(roc_auc_score(Ytest, Ypred_prob))

Classification Report (MLP Classifier):
              precision    recall  f1-score   support

           0       0.84      0.94      0.89      4548
           1       0.63      0.37      0.47      1266

    accuracy                           0.82      5814
   macro avg       0.74      0.66      0.68      5814
weighted avg       0.80      0.82      0.80      5814


Confusion Matrix (MLP Classifier):
[[4276  272]
 [ 794  472]]

ROC-AUC Score (MLP Classifier):
0.7490782886701929


In [31]:
mlp = MLPClassifier()
params = {'hidden_layer_sizes':[50]}
params['activation']=['logistic', 'tanh', 'relu']
params['solver']=['lbfgs', 'sgd', 'adam']
params['max_iter']=[75]
params['learning_rate']=['adaptive']
gclf = GridSearchCV(mlp, params, scoring='accuracy', n_jobs=-1)
gclf.fit(Xtrain, Ytrain)


/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (

GridSearchCV(estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [50],
                         'learning_rate': ['adaptive'], 'max_iter': [75],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             scoring='accuracy')

In [32]:
Ypred2 = gclf.best_estimator_.predict(Xtest)
Ypred_prob2 = gclf.best_estimator_.predict_proba(Xtest)[:, 1]

# Print Results
print("Classification Report (MLP Classifier):")
print(classification_report(Ytest, Ypred2))

print("\nConfusion Matrix (MLP Classifier):")
print(confusion_matrix(Ytest, Ypred2))

print("\nROC-AUC Score (MLP Classifier):")
print(roc_auc_score(Ytest, Ypred_prob2))

Classification Report (MLP Classifier):
              precision    recall  f1-score   support

           0       0.85      0.95      0.89      4548
           1       0.67      0.38      0.49      1266

    accuracy                           0.82      5814
   macro avg       0.76      0.66      0.69      5814
weighted avg       0.81      0.82      0.81      5814


Confusion Matrix (MLP Classifier):
[[4313  235]
 [ 784  482]]

ROC-AUC Score (MLP Classifier):
0.7603109399336688


In [35]:
gclf.best_params_

{'activation': 'logistic',
 'hidden_layer_sizes': 50,
 'learning_rate': 'adaptive',
 'max_iter': 75,
 'solver': 'adam'}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b862e48b-4303-4af0-9e88-f75fc6e5aab5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>